In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/telegram-spam-or-ham/dataset.csv


In [3]:
sms_df=pd.read_csv("../input/telegram-spam-or-ham/dataset.csv")
sms_df.head()

,text_type,text
0,spam,naturally irresistible your corporate identity...
1,spam,the stock trading gunslinger fanny is merrill ...
2,spam,unbelievable new homes made easy im wanting to...
3,spam,4 color printing special request additional in...
4,spam,do not have money get software cds from here s...


Labeling spam as **0** and ham as **1**

In [4]:
sms_df["text_type"]=sms_df["text_type"].map({"spam":0,"ham":1})


In [5]:
## cleaning the datasets
sms_df.isna().sum()
sms_df.fillna({"text_type":0},inplace=True)

In [6]:
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
X=sms_df["text"]
y=sms_df["text_type"]
vectorizer=TfidfVectorizer(ngram_range=(1,2),stop_words="english",max_features=5000)
## converting text data in numeric form 
X_train=vectorizer.fit_transform(X)

X_train,X_test,y_train,y_test=train_test_split(X_train,y,test_size=0.25,random_state=77)

In [7]:
print(vectorizer.get_feature_names_out())


['00' '00 00' '00 12' ... '𝚊𝚗𝚍' '𝚝𝚘' '𝚢𝚘𝚞']


In [8]:
## training the model 
lr=LogisticRegression().fit(X_train,y_train)
print("Training score : {:.2f}".format(lr.score(X_train,y_train)))
print("Testing score : {:.2f}".format(lr.score(X_test,y_test)))

Training score : 0.95
Testing score : 0.93


In [25]:
def predict_spam(text):
    text_transform = vectorizer.transform([text])
    prob = lr.predict_proba(text_transform)  # Get spam probability
    print("Spam Probability: {:.2f}".format(prob[0][0]))
    return "Spam" if prob[0][0] > 0.5 else "Not Spam"

In [27]:
message="Urgent: Your account has been compromised. Click here [fake-security.com] to reset your password now."
predict_spam(message)

Spam Probability: 0.77


'Spam'

In [26]:
message="Your order #56789 has been shipped and is expected to arrive on March 8. Track your package at [tracking-link.com]."
predict_spam(message)

Spam Probability: 0.33


'Not Spam'